In [14]:
import numpy as np

In [15]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [16]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [17]:

print("훈련 이미지 :",  mnist.train.images.shape)
print("훈련 라벨:",  mnist.train.labels.shape)
print("테스트 이미지 : ", mnist.test.images.shape)
print("테스트 라벨 : ", mnist.test.labels.shape)
print("검증 이미지 : ", mnist.validation.images.shape)
print("검증 라벨 : ", mnist.validation.labels.shape)
print('\n')


훈련 이미지 : (55000, 784)
훈련 라벨: (55000, 10)
테스트 이미지 :  (10000, 784)
테스트 라벨 :  (10000, 10)
검증 이미지 :  (5000, 784)
검증 라벨 :  (5000, 10)




In [18]:
X = tf.placeholder(tf.float32, [None, 24*24], name="X")
X_img = tf.reshape(X, [-1,28,28,1])
y = tf.placeholder(tf.float32, [None, 10], name="y")


In [19]:

W1 = tf.get_variable('W1', shape=[5, 5, 1, 32])
# shape (5, 5, 1, 32)

L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
# shape (?, 28, 28, 32)

L1 = tf.nn.relu(L1)
# shape (?, 28, 28, 32)

L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
# shape (?, 14, 14, 32)

ValueError: Variable W1 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 2023, in variable_v2
    shared_name=shared_name, name=name)


In [7]:

W2 = tf.get_variable('W2', shape=[5, 5, 32, 64])
# shape (5, 5, 32, 64)

L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
# shape (?, 14, 14, 64)

L2 = tf.nn.relu(L2)
# shape (?, 14, 14, 64)

L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
# shape (?, 7, 7, 64)

In [13]:

# fully-connected를 위한 matrix flatten
L2 = tf.layers.flatten(L2)

# fully-connected
W3 = tf.get_variable('W3', shape=[7*7*64, 1024], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1024], stddev=0.01))
L3 = tf.matmul(L2, W3) + b3
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable('W4', shape=[1024, 10], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([10], stddev=0.01))
logit = tf.matmul(L3, W4) + b4


ValueError: Variable W3 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\Users\i0109\anaconda3\envs\Test\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 2023, in variable_v2
    shared_name=shared_name, name=name)


In [10]:

learning_rate = 0.01
batch_size = 1000
num_epoch = 15
keep_prob = tf.placeholder(tf.float32)


In [12]:

hypothesis = tf.nn.softmax(logit)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# accuracy 측정
predicted = tf.argmax(hypothesis, axis=1)
actual = tf.argmax(Y, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, actual), tf.float32))

NameError: name 'logit' is not defined

In [11]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_batch = int(mnist.train.num_examples / batch_size)
    for epoch in range(num_epoch):
        avg_cost = 0
        for b in range(num_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.4})
            avg_cost += cost_val / num_batch
        print("epoch {0}, cost = {1:.5f}".format(epoch, cost_val))
    accuracy_val = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}) 
    print("result, accuracy = {0:.5f}".format(accuracy_val))


NameError: name 'cost' is not defined